In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from collections import Counter, defaultdict
from pathlib import Path
import time

import cv2
import numpy as np
import pandas as pd
import torch
import tqdm
from Levenshtein import distance
from matplotlib import pyplot as plt
from scipy.special import softmax
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
import torch.nn.functional as fnn

from detection_utils import PlateImageAdjuster, PlateImageExtractor, build_mask, get_rectangular_box
from recognition import CRNN, RecognitionDataset, LanguageModel, beam_search
from recognition_utils import collate_fn_recognition, decode, normalize_text, Resize

%matplotlib inline

In [3]:
normalizer = PlateImageAdjuster()
extractor = PlateImageExtractor()

## Prepare OCR dataset

In [4]:
path_data = Path('data')
path_ocr_dataset = Path('ocr_data')
path_ocr_dataset.mkdir(parents=True, exist_ok=True)

plates_filename = path_data / 'train.json'
with open(plates_filename) as f:
    json_data = json.load(f)
    
for sample in tqdm.tqdm(json_data):
    if sample['file'] == 'train/25632.bmp':
        continue
    file_path = path_data / sample['file']
    image = cv2.imread(str(file_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    for plate in sample['nums']:
        box = plate['box']
        text = plate['text']
        mask = build_mask(box, image)
        plate_img = extractor(image, mask, np.array(box))
        plate_img = normalizer(plate_img)
        text = normalize_text(text)
        file_path = path_ocr_dataset / ''.join([text, '.png'])
        cv2.imwrite(str(file_path), plate_img)
        
        # save also bboxes
        file_path = path_ocr_dataset / ''.join([text, '_bbox.png'])
        raw_box = get_rectangular_box(box)
        plate_bbox = image[raw_box[1]:raw_box[3], raw_box[0]:raw_box[2], :]
        plate_bbox = normalizer(plate_bbox)
        cv2.imwrite(str(file_path), plate_bbox)

100%|██████████| 25633/25633 [14:53<00:00, 28.69it/s]


## Build the model

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
crnn = CRNN(rnn_bidirectional=True)

In [5]:
crnn.to(device)
num_epochs = 20
batch_size = 64
num_workers = 4
optimizer = torch.optim.Adam(crnn.parameters(), lr=3e-4, amsgrad=True, weight_decay=1e-5)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=1/np.sqrt(10), patience=2,
                                                          verbose=True, threshold=1e-3)

In [6]:
transformations = transforms.Compose([
    Resize(),
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])
                    ])

train_ocr_dataset = RecognitionDataset('ocr_data', transformations, crnn.alphabet, 'train', add_generated=True)
val_ocr_dataset = RecognitionDataset('ocr_data', transformations, crnn.alphabet, 'val')

In [8]:
train_dataloader = torch.utils.data.DataLoader(train_ocr_dataset, 
                                               batch_size=batch_size, shuffle=True,
                                               num_workers=num_workers, pin_memory=True, 
                                               drop_last=False, collate_fn=collate_fn_recognition)
val_dataloader = torch.utils.data.DataLoader(val_ocr_dataset, 
                                             batch_size=1, shuffle=False,
                                             num_workers=num_workers, pin_memory=True, 
                                             drop_last=True, collate_fn=collate_fn_recognition)

In [9]:
experiment_name = 'Recognition_model_with_generated_test'
writer = SummaryWriter(log_dir=f'tb_logs/{experiment_name}')

In [10]:
best_loss = np.inf
prev_lr = optimizer.param_groups[0]['lr']

for i, epoch in enumerate(range(num_epochs)):
    epoch_losses = []
    levensteint_losses = []
    
    # Если поменялась lr - загружаем лучшую модель
    if optimizer.param_groups[0]['lr'] < prev_lr:
        prev_lr = optimizer.param_groups[0]['lr']
        with open(f'{experiment_name}.pth', 'rb') as fp:
            state_dict = torch.load(fp, map_location="cpu")
        crnn.load_state_dict(state_dict)
        crnn.to(device)
    
    crnn.train()
    for j, b in enumerate(tqdm.tqdm(train_dataloader, total=len(train_dataloader))):
        images = b["image"].to(device)
        seqs_gt = b["seq"]
        seq_lens_gt = b["seq_len"]

        seqs_pred = crnn(images).cpu()
        log_probs = fnn.log_softmax(seqs_pred, dim=2)
        seq_lens_pred = torch.Tensor([seqs_pred.size(0)] * seqs_pred.size(1)).int()
        
        texts_pred = decode(seqs_pred, crnn.alphabet)
        texts_gt = b["text"]
        levensteint_losses.extend([distance(pred, gt) for pred, gt in zip(texts_pred, texts_gt)])

        loss = fnn.ctc_loss(log_probs=log_probs,  # (T, N, C)
                            targets=seqs_gt,  # N, S or sum(target_lengths)
                            input_lengths=seq_lens_pred,  # N
                            target_lengths=seq_lens_gt)  # N

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())
    print(f'Train {i + 1}, {np.mean(epoch_losses)}')
    print(f'Train {i + 1} Levenstein, {np.mean(levensteint_losses)}')
    writer.add_scalar('Recognition/Train/loss', np.mean(epoch_losses), i)
    time.sleep(0.5)
    
    epoch_losses = []
    levensteint_losses = []
    crnn.eval()
    for j, b in enumerate(tqdm.tqdm(val_dataloader, total=len(val_dataloader))):
        images = b["image"].to(device)
        seqs_gt = b["seq"]
        seq_lens_gt = b["seq_len"]

        seqs_pred = crnn(images).cpu()
        log_probs = fnn.log_softmax(seqs_pred, dim=2)
        seq_lens_pred = torch.Tensor([seqs_pred.size(0)] * seqs_pred.size(1)).int()
        
        texts_pred = decode(seqs_pred, crnn.alphabet)
        texts_gt = b["text"]
        levensteint_losses.extend([distance(pred, gt) for pred, gt in zip(texts_pred, texts_gt)])

        loss = fnn.ctc_loss(log_probs=log_probs,  # (T, N, C)
                            targets=seqs_gt,  # N, S or sum(target_lengths)
                            input_lengths=seq_lens_pred,  # N
                            target_lengths=seq_lens_gt)  # N

        epoch_losses.append(loss.item())
        
        if best_loss > epoch_losses[-1]:
            best_loss = epoch_losses[-1]
            with open(f'{experiment_name}.pth', 'wb') as fp:
                torch.save(crnn.state_dict(), fp)
        
    lr_scheduler.step(np.mean(levensteint_losses))
    print(f'Valid {i + 1}, {np.mean(epoch_losses)}')
    print(f'Valid {i + 1} Levenstein, {np.mean(levensteint_losses)}')
    writer.add_scalar('Recognition/Valid/loss', np.mean(epoch_losses), i)
    time.sleep(0.5)

100%|██████████| 1599/1599 [07:30<00:00,  3.55it/s]


Train 1, 0.3902352593793738
Train 1 Levenstein, 0.8286814809021498


100%|██████████| 2370/2370 [00:19<00:00, 121.09it/s]


Valid 1, 0.06148304197738804
Valid 1 Levenstein, 0.11012658227848102


100%|██████████| 1599/1599 [07:30<00:00,  3.55it/s]


...

Valid 19, 0.034593770058170714
Valid 19 Levenstein, 0.052742616033755275


100%|██████████| 1599/1599 [07:31<00:00,  3.54it/s]


Train 20, 0.042898616077305
Train 20 Levenstein, 0.10439050142244034


100%|██████████| 2370/2370 [00:17<00:00, 137.56it/s]


Epoch    20: reducing learning rate of group 0 to 3.0000e-06.
Valid 20, 0.034873423276246596
Valid 20 Levenstein, 0.053164556962025315


In [9]:
with open(f'{experiment_name}.pth', 'rb') as fp:
    state_dict = torch.load(fp, map_location="cpu")
crnn = CRNN(rnn_bidirectional=True)
crnn.load_state_dict(state_dict)
crnn.to(device)
crnn.eval()
print('Model loaded!')

Model loaded!
